In [0]:
print("Lets start over")

Lets start over


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("rockstar").getOrCreate()

In [0]:
dt=spark.read.option("header",True).csv("/FileStore/tables/MS_Dhoni_ODI_record_test.csv")

In [0]:
dt.printSchema()

root
 |-- score: string (nullable = true)
 |-- runs_scored: string (nullable = true)
 |-- balls_faced: string (nullable = true)
 |-- strike_rate: string (nullable = true)
 |-- fours: string (nullable = true)
 |-- sixes: string (nullable = true)
 |-- opposition: string (nullable = true)
 |-- ground: string (nullable = true)
 |-- date: string (nullable = true)
 |-- odi_number: string (nullable = true)



In [0]:
dt.count()

Out[5]: 350

In [0]:
dt.take(2)

Out[6]: [Row(score='0', runs_scored='0', balls_faced='1', strike_rate='0', fours='0', sixes='0', opposition='v Bangladesh', ground='Chattogram', date='12/23/2004', odi_number='ODI # 2199'),
 Row(score='12', runs_scored='12', balls_faced='11', strike_rate='109.09', fours='2', sixes='0', opposition='v Bangladesh', ground='Dhaka', date='12/26/2004', odi_number='ODI # 2201')]

In [0]:
total_fours=dt.agg({'fours':'sum'})

In [0]:
print(f"Total fours {total_fours.show()}")

+----------+
|sum(fours)|
+----------+
|     826.0|
+----------+

Total fours None


In [0]:
#matches played in Dhaka
dt.filter(dt.ground=='Dhaka').show()

+-----+-----------+-----------+-----------+-----+-----+------------+------+----------+----------+
|score|runs_scored|balls_faced|strike_rate|fours|sixes|  opposition|ground|      date|odi_number|
+-----+-----------+-----------+-----------+-----+-----+------------+------+----------+----------+
|   12|         12|         11|     109.09|    2|    0|v Bangladesh| Dhaka|12/26/2004|ODI # 2201|
|   7*|          7|          2|        350|    0|    1|v Bangladesh| Dhaka|12/27/2004|ODI # 2202|
|  91*|         91|        106|      85.84|    7|    0|v Bangladesh| Dhaka| 5/10/2007|ODI # 2582|
|   36|         36|         54|      66.66|    4|    0|v Bangladesh| Dhaka| 5/12/2007|ODI # 2583|
|   24|         24|         23|     104.34|    5|    0|  v Pakistan| Dhaka| 6/10/2008|ODI # 2705|
|  DNB|          -|          -|          -|    -|    -|v Bangladesh| Dhaka| 6/12/2008|ODI # 2706|
|   64|         64|         59|     108.47|    3|    2|  v Pakistan| Dhaka| 6/14/2008|ODI # 2707|
|   37|         37| 

In [0]:
#count of matches played against opposition
from pyspark.sql.functions import *
dt.groupBy("opposition").count().orderBy(col('count').desc()).show()

+--------------+-----+
|    opposition|count|
+--------------+-----+
|   v Sri Lanka|   67|
|   v Australia|   55|
|     v England|   48|
| v West Indies|   39|
|v South Africa|   37|
|    v Pakistan|   36|
| v New Zealand|   28|
|  v Bangladesh|   21|
|    v Zimbabwe|    6|
|   v Africa XI|    3|
|     v Ireland|    2|
|   v Hong Kong|    2|
| v Afghanistan|    2|
| v Netherlands|    1|
|     v Bermuda|    1|
|      v U.A.E.|    1|
|    v Scotland|    1|
+--------------+-----+



In [0]:
dt.first()

Out[11]: Row(score='0', runs_scored='0', balls_faced='1', strike_rate='0', fours='0', sixes='0', opposition='v Bangladesh', ground='Chattogram', date='12/23/2004', odi_number='ODI # 2199')

In [0]:
dt.filter(dt.score== 0).select('score','opposition','date').show()

+-----+--------------+----------+
|score|    opposition|      date|
+-----+--------------+----------+
|    0|  v Bangladesh|12/23/2004|
|    0|   v Sri Lanka| 11/6/2005|
|    0|  v Bangladesh| 3/17/2007|
|    0|   v Sri Lanka| 3/23/2007|
|    0|v South Africa| 6/29/2007|
|    0|   v Australia|10/20/2010|
|    0|     v England| 6/23/2013|
|    0|   v Australia| 1/20/2016|
|    0|   v Hong Kong| 9/18/2018|
|    0|   v Australia|  3/5/2019|
+-----+--------------+----------+



In [0]:
#add new col
dt.withColumn("country",lit('India')).show()

+-----+-----------+-----------+-----------+-----+-----+-------------+-------------+----------+----------+-------+
|score|runs_scored|balls_faced|strike_rate|fours|sixes|   opposition|       ground|      date|odi_number|country|
+-----+-----------+-----------+-----------+-----+-----+-------------+-------------+----------+----------+-------+
|    0|          0|          1|          0|    0|    0| v Bangladesh|   Chattogram|12/23/2004|ODI # 2199|  India|
|   12|         12|         11|     109.09|    2|    0| v Bangladesh|        Dhaka|12/26/2004|ODI # 2201|  India|
|   7*|          7|          2|        350|    0|    1| v Bangladesh|        Dhaka|12/27/2004|ODI # 2202|  India|
|    3|          3|          7|      42.85|    0|    0|   v Pakistan|        Kochi|  4/2/2005|ODI # 2235|  India|
|  148|        148|        123|     120.32|   15|    4|   v Pakistan|Visakhapatnam|  4/5/2005|ODI # 2236|  India|
|   28|         28|         24|     116.66|    5|    0|   v Pakistan|   Jamshedpur|  4/9

In [0]:
dt.filter(dt.score != 'DNB').agg({'date':'min'}).show()

+---------+
|min(date)|
+---------+
|1/11/2010|
+---------+



In [0]:
dt.filter(dt.date == '1/11/2010').show()

+-----+-----------+-----------+-----------+-----+-----+------------+------+---------+----------+
|score|runs_scored|balls_faced|strike_rate|fours|sixes|  opposition|ground|     date|odi_number|
+-----+-----------+-----------+-----------+-----+-----+------------+------+---------+----------+
|   32|         32|         35|      91.42|    3|    0|v Bangladesh| Dhaka|1/11/2010|ODI # 2942|
+-----+-----------+-----------+-----------+-----+-----+------------+------+---------+----------+



In [0]:
#dt.filter((dt.score != 'DNB') & (dt.score != 'TDNB')).agg({'date':'max'}).show()
#dt.agg({to_date(col('date'),'dd-mm-yyyy'):'max'}).show()
dt.withColumn("Ddate",to_date(col('date'),'dd-MM-yyyy')).show()

+-----+-----------+-----------+-----------+-----+-----+-------------+-------------+----------+----------+-----+
|score|runs_scored|balls_faced|strike_rate|fours|sixes|   opposition|       ground|      date|odi_number|Ddate|
+-----+-----------+-----------+-----------+-----+-----+-------------+-------------+----------+----------+-----+
|    0|          0|          1|          0|    0|    0| v Bangladesh|   Chattogram|12/23/2004|ODI # 2199| null|
|   12|         12|         11|     109.09|    2|    0| v Bangladesh|        Dhaka|12/26/2004|ODI # 2201| null|
|   7*|          7|          2|        350|    0|    1| v Bangladesh|        Dhaka|12/27/2004|ODI # 2202| null|
|    3|          3|          7|      42.85|    0|    0|   v Pakistan|        Kochi|  4/2/2005|ODI # 2235| null|
|  148|        148|        123|     120.32|   15|    4|   v Pakistan|Visakhapatnam|  4/5/2005|ODI # 2236| null|
|   28|         28|         24|     116.66|    5|    0|   v Pakistan|   Jamshedpur|  4/9/2005|ODI # 2237

In [0]:
dt.filter(dt.date == '9/9/2011').show()

+-----+-----------+-----------+-----------+-----+-----+----------+--------+--------+----------+
|score|runs_scored|balls_faced|strike_rate|fours|sixes|opposition|  ground|    date|odi_number|
+-----+-----------+-----------+-----------+-----+-----+----------+--------+--------+----------+
|   69|         69|        103|      66.99|    5|    0| v England|The Oval|9/9/2011|ODI # 3189|
+-----+-----------+-----------+-----------+-----+-----+----------+--------+--------+----------+



In [0]:
total_runs=dt.agg({'runs_scored':'sum'}).show()

+----------------+
|sum(runs_scored)|
+----------------+
|         10773.0|
+----------------+



In [0]:
total_ball_faced=dt.agg({'balls_faced':'sum'}).show()

+----------------+
|sum(balls_faced)|
+----------------+
|         12303.0|
+----------------+



In [0]:
career_strike_rate=dt.agg({'strike_rate':'avg'}).show()

+-----------------+
| avg(strike_rate)|
+-----------------+
|87.93996632996635|
+-----------------+



In [0]:
dt.filter(dt.score.contains("*")).count()

Out[25]: 84

In [0]:
dt.filter(dt.runs_scored >= 100).count()

Out[30]: 10

In [0]:
runs_scored_50s=dt.filter(dt.runs_scored >= 50).count()
print(runs_scored_50s)

83


In [0]:
dt.filter((dt.score != 'DNB') & (dt.score != 'TDNB')).count()

Out[37]: 297

In [0]:
dt.count()

Out[38]: 350

In [0]:
dt.take(2)

Out[48]: [Row(score='0', runs_scored='0', balls_faced='1', strike_rate='0', fours='0', sixes='0', opposition='v Bangladesh', ground='Chattogram', date='12/23/2004', odi_number='ODI # 2199'),
 Row(score='12', runs_scored='12', balls_faced='11', strike_rate='109.09', fours='2', sixes='0', opposition='v Bangladesh', ground='Dhaka', date='12/26/2004', odi_number='ODI # 2201')]

In [0]:
left_dt=dt.select('odi_number','date','ground','opposition')
display(left_dt)

odi_number,date,ground,opposition
ODI # 2199,12/23/2004,Chattogram,v Bangladesh
ODI # 2201,12/26/2004,Dhaka,v Bangladesh
ODI # 2202,12/27/2004,Dhaka,v Bangladesh
ODI # 2235,4/2/2005,Kochi,v Pakistan
ODI # 2236,4/5/2005,Visakhapatnam,v Pakistan
ODI # 2237,4/9/2005,Jamshedpur,v Pakistan
ODI # 2238,4/12/2005,Ahmedabad,v Pakistan
ODI # 2239,4/15/2005,Kanpur,v Pakistan
ODI # 2240,4/17/2005,Delhi,v Pakistan
ODI # 2262,7/30/2005,Dambulla,v Sri Lanka


In [0]:
right_dt=dt.select('odi_number','score','runs_scored','balls_faced','fours','sixes')
display(right_dt)

odi_number,score,runs_scored,balls_faced,fours,sixes
ODI # 2199,0,0,1,0,0
ODI # 2201,12,12,11,2,0
ODI # 2202,7*,7,2,0,1
ODI # 2235,3,3,7,0,0
ODI # 2236,148,148,123,15,4
ODI # 2237,28,28,24,5,0
ODI # 2238,47,47,64,4,1
ODI # 2239,11,11,16,1,0
ODI # 2240,24,24,38,2,0
ODI # 2262,2,2,7,0,0


In [0]:
join_dt=left_dt.join(right_dt,left_dt.odi_number==right_dt.odi_number,how='inner')

In [0]:
join_dt.filter((join_dt.score != 'DNB') & (join_dt.score != 'TDNB')).count()

Out[58]: 297